In [20]:
import pandas as pd
import numpy as np
from trino.dbapi import connect 

In [21]:
def read_from_hive(sql_script):
    conn = connect(
            host='presto.bstis.com',
            port=8080,
            user='hadoop',
            catalog='hive',
            #schema='default'
            )
    cur = conn.cursor()
    cur.execute(sql_script)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows,columns=columns)
    return df

### Set Period

In [22]:
curr_month = (pd.to_datetime(('2022-11-01'))).strftime('%Y-%m-%d')

In [23]:
stop_monthsdata = curr_month
start_monthsdata = (pd.to_datetime(curr_month) - pd.DateOffset(months=36)).strftime('%Y-%m-%d')
stop_date = (pd.to_datetime(curr_month) + pd.DateOffset(months=1) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date = (pd.to_datetime(stop_date) - pd.DateOffset(months=12) + pd.DateOffset(days=1) ).strftime('%Y-%m-%d')

In [24]:
sSQL = '''
select
division as tenantid, 
carrier,
exchange,
count(distinct month) as months,
max(month) as MaxMonth

from hive.trinet.elig

where  month <= date('{stop_monthsdata}')
 AND month >= date('{start_monthsdata}')
 and status <> 'TERMED'
 and carrier <> 'Med_Waived'

group by 1,2,3
'''.format(stop_monthsdata=stop_monthsdata, start_monthsdata=start_monthsdata)

df_monthsdata = read_from_hive(sSQL)
del sSQL

df_monthsdata.to_csv(f'Elig Data/MonthsData_{stop_monthsdata.replace("-", "_")}.csv',index=False)

In [25]:
sSQL = '''
SELECT division AS tenantid,
       tag_tpa AS carrier,
       tag_exchange AS exchange,
       personid,
       drg_code,
       drg_code_description,
       SUM(amtallowed) AS amtallowed,
       sum(amtpaid) AS amtpaid
       FROM hive.trinet.claims
WHERE paiddate >= DATE ('{start_date}')
AND   paiddate <= DATE ('{stop_date}')
AND   drg_code_description LIKE '%TRANSPLANT%'
GROUP BY 1,2,3,4,5,6
'''.format(start_date=start_date, stop_date=stop_date)

df_transplants = read_from_hive(sSQL)
del sSQL

df_transplants.to_csv(f'Claims Data/Transplants_{curr_month.replace("-", "_")}.csv',index=False)